In [1]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import os

secret = os.environ.get("SPARK_AUTH_SECRET")
conf = SparkConf()
conf.set("spark.authenticate", "true")
conf.set("spark.driver.extraJavaOptions", f"-Dspark.authenticate.secret={secret}")

spark = SparkSession.builder.config(conf=conf)\
    .appName("merge_silver") \
    .master("spark://node1.dw.felicity.net.bd:7077,node2.dw.felicity.net.bd:7077")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/11 03:17:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/06/11 03:17:17 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/06/11 03:17:18 WARN S3ABlockOutputStream: Application invoked the Syncable API against stream writing to spark-events/app-20250611031712-0037.inprogress. This is unsupported


In [2]:
spark.sql(f"""
MERGE INTO silver.dim_location AS target
USING bronze.geo_location AS source
ON target.location_id = source.location_id
WHEN MATCHED THEN
  UPDATE SET
    target.country = source.country,
    target.state = source.state,
    target.city = source.city,
    target.postal_code = source.postal_code,
    target.load_timestamp = source.load_timestamp
WHEN NOT MATCHED THEN
  INSERT (location_id, country, state, city, postal_code, load_timestamp)
  VALUES (source.location_id, source.country, source.state, source.city, source.postal_code, source.load_timestamp);
""");

25/06/11 03:17:23 WARN HiveConf: HiveConf of name hive.metastore.ssl.need.client.auth does not exist


In [4]:
spark.sql(f"""
MERGE INTO silver.dim_customer AS target
USING bronze.customer AS source
ON target.customer_id = source.customer_id
WHEN MATCHED THEN
  UPDATE SET
    target.full_name = source.full_name,
    target.email = source.email,
    target.phone_number = source.phone_number,
    target.location_id = source.location_id,
    target.created_at = source.created_at,
    target.load_timestamp = source.load_timestamp
WHEN NOT MATCHED THEN
  INSERT (
    customer_id, full_name, email, phone_number,
    location_id, created_at, load_timestamp
  )
  VALUES (
    source.customer_id, source.full_name, source.email, source.phone_number,
    source.location_id, source.created_at, source.load_timestamp
  );


""");

25/06/11 03:43:48 WARN HiveConf: HiveConf of name hive.metastore.ssl.need.client.auth does not exist


In [5]:
spark.sql(f"""
MERGE INTO silver.dim_product AS target
USING bronze.product AS source
ON target.product_id = source.product_id
WHEN MATCHED THEN
  UPDATE SET
    target.product_name = source.product_name,
    target.description = source.description,
    target.category = source.category,
    target.price = source.price,
    target.in_stock = source.in_stock,
    target.created_at = source.created_at,
    target.load_timestamp = source.load_timestamp
WHEN NOT MATCHED THEN
  INSERT (
    product_id, product_name, description, category,
    price, in_stock, created_at, load_timestamp
  )
  VALUES (
    source.product_id, source.product_name, source.description, source.category,
    source.price, source.in_stock, source.created_at, source.load_timestamp
  );


""");

In [10]:
spark.sql(f"""
CREATE OR REPLACE TEMPORARY VIEW sales_fact_staging AS
SELECT
    oi.order_item_id,
    oi.order_id,
    so.customer_id,
    oi.product_id,
    cu.location_id,
    so.order_date,
    so.updated_at,
    oi.quantity,
    oi.unit_price,
    oi.total_price,
    so.status,
    oi.load_timestamp
FROM bronze.order_item oi
JOIN bronze.sales_order so ON oi.order_id = so.order_id
JOIN bronze.customer cu ON so.customer_id = cu.customer_id;

""");

In [11]:
spark.sql(f"""
MERGE INTO silver.fact_sales AS target
USING sales_fact_staging AS source
ON target.order_item_id = source.order_item_id
WHEN MATCHED THEN
  UPDATE SET
    target.order_id = source.order_id,
    target.customer_id = source.customer_id,
    target.product_id = source.product_id,
    target.location_id = source.location_id,
    target.order_date = source.order_date,
    target.updated_at = source.updated_at,
    target.quantity = source.quantity,
    target.unit_price = source.unit_price,
    target.total_price = source.total_price,
    target.status = source.status,
    target.load_timestamp = source.load_timestamp
WHEN NOT MATCHED THEN
  INSERT (
    order_item_id, order_id, customer_id, product_id,
    location_id, order_date, updated_at,
    quantity, unit_price, total_price, status, load_timestamp
  )
  VALUES (
    source.order_item_id, source.order_id, source.customer_id, source.product_id,
    source.location_id, source.order_date, source.updated_at,
    source.quantity, source.unit_price, source.total_price, source.status, source.load_timestamp
  );


""");

25/06/11 03:58:19 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
